In [1]:
!mkdir results

In [2]:
import torch
import torchvision  #비전관련 library
import torchvision.transforms as transforms  #데이터 조작
import torch.nn as nn  
import torch.nn.functional as F  #Neural Network 구조 construct
import torch.optim as optim  #최적화
import argparse  #hyper-parameter셋팅
import numpy as np  #배열연산
import time #시간계산
from copy import deepcopy # Add Deepcopy for args
import seaborn as sns  #시각화
import matplotlib.pyplot as plt #시각화

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# train data를 4:1로 val로 나눔

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainset, valset = torch.utils.data.random_split(trainset, [40000, 10000])
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

partition = {'train': trainset, 'val':valset, 'test':testset}

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
class MLP(nn.Module):
    def __init__(self, in_dim, out_dim, hid_dim, n_layer, act, dropout, use_bn, use_xavier):
        super(MLP, self).__init__()
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.hid_dim = hid_dim
        self.n_layer = n_layer
        self.act = act
        self.dropout = dropout
        self.use_bn = use_bn
        self.use_xavier = use_xavier
        
        # ====== Create Linear Layers ====== #
        self.fc1 = nn.Linear(self.in_dim, self.hid_dim)
        
        self.linears = nn.ModuleList()
        self.bns = nn.ModuleList()
        for i in range(self.n_layer-1):
            self.linears.append(nn.Linear(self.hid_dim, self.hid_dim))
            if self.use_bn:
                self.bns.append(nn.BatchNorm1d(self.hid_dim))
                
        self.fc2 = nn.Linear(self.hid_dim, self.out_dim)
        
        # ====== Create Activation Function ====== #
        if self.act == 'relu':
            self.act = nn.ReLU()
        elif self.act == 'tanh':
            self.act == nn.Tanh()
        elif self.act == 'sigmoid':
            self.act = nn.Sigmoid()
        else:
            raise ValueError('no valid activation function selected!')
        
        # ====== Create Regularization Layer ======= #
        self.dropout = nn.Dropout(self.dropout)
        if self.use_xavier:
            self.xavier_init()
          
    def forward(self, x):
        x = self.act(self.fc1(x))
        for i in range(len(self.linears)):
            x = self.act(self.linears[i](x))
            if self.use_bn:
                x = self.bns[i](x)
            x = self.dropout(x)
        x = self.fc2(x)
        return x
    
    def xavier_init(self):
        for linear in self.linears:
            nn.init.xavier_normal_(linear.weight)
            linear.bias.data.fill_(0.01)

In [5]:
cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

In [6]:
class CNN(nn.Module):
    
    def __init__(self, model_code, in_channels, out_dim, act, use_bn): 
        super(CNN, self).__init__() 
        
        if act == 'relu':
            self.act = nn.ReLU()
        elif act == 'sigmoid':
            self.act = nn.Sigmoid()
        elif act == 'tanh':
            self.act = nn.TanH()
        else:
            raise ValueError("Not a valid activation function code")
        
        self.layers = self._make_layers(model_code, in_channels, use_bn)   # private function 여기서만 사용할 함수
        self.classifer = nn.Sequential(nn.Linear(512, 256), # fully connected layer
                                       self.act,
                                       nn.Linear(256, out_dim))   # linear layer를 (여러개 쌓음 fc마지막에 두개 붙이듯이) / list 형식 아니면 *안붙여도 됨 / linear 후엔 activation, 마지막엔 softmax 들어가기전
        
    def forward(self, x):
        x = self.layers(x)
        # # nn.Sequential 안했을 경우
        # for layer in layers:
        #   x = layer(x)
        x = x.view(x.size(0), -1)
        x = self.classifer(x)
        return x
        
    def _make_layers(self, model_code, in_channels, use_bn):  # 함수 부를 때도 넣어줘야함(self.layers) / 생성자에서 받아와야함(__init__)
        layers = []
        for x in cfg[model_code]:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels=in_channels,
                                     out_channels=x,
                                     kernel_size=3,
                                     stride=1,
                                     padding=1)]
                if use_bn:
                    layers += [nn.BatchNorm2d(x)]
                layers += [self.act]
                in_channels = x
        return nn.Sequential(*layers)  # list에 layer object를 넣어주고 nn.Sequential을 하면 sub 모듈을 만들어주는 효과 / nn.Sequnetiail을 안하면  for문으로 하나씩 layer 넣어줘야함

In [7]:
# class CNN1(nn.Module):
    
#     def __init__(self): # construct
#         super(CNN1, self).__init__()       #상속 
        
#         # object 생성
#         self.conv1 = nn.Conv2d(in_channels = 3,
#                                out_channels = 64,
#                                kernel_size = 3,
#                                stride = 1,
#                                padding = 1)  # kernel_size/2로 하면 크기 유지됨
#         self.conv2 = nn.Conv2d(in_channels = 64,
#                                out_channels = 256,
#                                kernel_size = 5,
#                                stride = 1,
#                                padding = 2)
#         self.act = nn.ReLU()   # 파라미터가 없기 때문에 여러개를 만들어주지 않아도 됨
#         self.maxpool1 = nn.MaxPool2d(kernel_size = 2,
#                                      stride = 2)   # 절반으로 줄여줌 
#         self.fc = nn.Linear(65536, 10)  # 일자로 폈을 때 65536d / 10 class

#     def forward(self, x):   # 실제 사용
#         x = self.conv1(x)
#         x = self.act(x)
#         x = self.conv2(x)
#         x = self.act(x)
#         x = self.maxpool1(x)
#         x = x.view(x.size(0), -1)  # linear를 넣기 전에는 2d를 1d로 / x.size(0):batch size / -1:나머지 일자로
#         x = self.fc(x)
#         return x

# def dimension_check():
#     net = CNN1()
#     x = torch.randn(2, 3, 32, 32)
#     y = net(x)
#     print(y.size())

# dimension_check()

torch.Size([2, 10])


In [10]:
# class CNN2(nn.Module):
    
#     def __init__(self, model_code, in_channels): 
#         super(CNN2, self).__init__() 
        
#         self.layers = self._make_layers(model_code, in_channels)   # private function 여기서만 사용할 함수
#         self.classifer = nn.Sequential(nn.Linear(512, 256),
#                                        nn.ReLU(),
#                                        nn.Linear(256, 10))   # linear layer를 (여러개 쌓음 fc마지막에 두개 붙이듯이) / list 형식 아니면 *안붙여도 됨 / linear 후엔 activation, 마지막엔 softmax 들어가기전
        
#     def forward(self, x):
#         x = self.layers(x)
#         # # nn.Sequential 안했을 경우
#         # for layer in layers:
#         #   x = layer(x)
#         x = x.view(x.size(0), -1)
#         x = self.classifer(x)
#         return x
        
#     def _make_layers(self, model_code, in_channels):  # 함수 부를 때도 넣어줘야함(self.layers) / 생성자에서 받아와야함(__init__)
#         layers = []
#         for x in cfg[model_code]:
#             if x == 'M':
#                 layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
#             else:
#                 layers += [nn.Conv2d(in_channels=in_channels,
#                                      out_channels=x,
#                                      kernel_size=3,
#                                      stride=1,
#                                      padding=1)]
#                 layers += [nn.ReLU()]
#                 in_channels = x
#         return nn.Sequential(*layers)  # list에 layer object를 넣어주고 nn.Sequential을 하면 sub 모듈을 만들어주는 효과 / nn.Sequnetiail을 안하면  for문으로 하나씩 layer 넣어줘야함

# def dimension_check():
#     net = CNN2('VGG11', 3)
#     x = torch.randn(2, 3, 32, 32)
#     y = net(x)
#     print(y.size())

# dimension_check()

torch.Size([2, 10])


In [11]:
# 만든 CNN이 차원이 맞는지 확인
def dimension_check():
    net = CNN('VGG11', 3)
    x = torch.randn(2, 3, 32, 32)
    y = net(x)
    print(y.size())

In [12]:
def train(net, partition, optimizer, criterion, args):
    trainloader = torch.utils.data.DataLoader(partition['train'], 
                                              batch_size=args.train_batch_size, 
                                              shuffle=True, num_workers=2)
    net.train()
    optimizer.zero_grad()

    correct = 0
    total = 0
    train_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = net(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = train_loss / len(trainloader)
    train_acc = 100 * correct / total
    return net, train_loss, train_acc

In [13]:
def validate(net, partition, criterion, args):
    valloader = torch.utils.data.DataLoader(partition['val'], 
                                            batch_size=args.test_batch_size, 
                                            shuffle=False, num_workers=2)
    net.eval()

    correct = 0
    total = 0
    val_loss = 0 
    with torch.no_grad():
        for data in valloader:
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()
            outputs = net(images)

            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        val_loss = val_loss / len(valloader)
        val_acc = 100 * correct / total
    return val_loss, val_acc

In [14]:
def test(net, partition, args):
    testloader = torch.utils.data.DataLoader(partition['test'], 
                                             batch_size=args.test_batch_size, 
                                             shuffle=False, num_workers=2)
    net.eval()
    
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()

            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        test_acc = 100 * correct / total
    return test_acc

In [15]:
def experiment(partition, args):
  
    net = CNN(model_code = args.model_code,
              in_channels = args.in_channels,
              out_dim = args.out_dim,
              act = args.act,
              use_bn = args.use_bn)
    # net = CNN1()
    net.cuda()

    criterion = nn.CrossEntropyLoss()
    if args.optim == 'SGD':
        optimizer = optim.SGD(net.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'RMSprop':
        optimizer = optim.RMSprop(net.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'Adam':
        optimizer = optim.Adam(net.parameters(), lr=args.lr, weight_decay=args.l2)
    else:
        raise ValueError('In-valid optimizer choice')
    
    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []
        
    for epoch in range(args.epoch):  # loop over the dataset multiple times
        ts = time.time()
        net, train_loss, train_acc = train(net, partition, optimizer, criterion, args)
        val_loss, val_acc = validate(net, partition, criterion, args)
        te = time.time()
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accs.append(train_acc)
        val_accs.append(val_acc)
        
        print('Epoch {}, Acc(train/val): {:2.2f}/{:2.2f}, Loss(train/val) {:2.2f}/{:2.2f}. Took {:2.2f} sec'.format(epoch, train_acc, val_acc, train_loss, val_loss, te-ts))
        
    test_acc = test(net, partition, args)    
    
    result = {}
    result['train_losses'] = train_losses
    result['val_losses'] = val_losses
    result['train_accs'] = train_accs
    result['val_accs'] = val_accs
    result['train_acc'] = train_acc
    result['val_acc'] = val_acc
    result['test_acc'] = test_acc
    return vars(args), result

In [16]:
import hashlib
import json
from os import listdir
from os.path import isfile, join
import pandas as pd

def save_exp_result(setting, result):
    exp_name = setting['exp_name']
    del setting['epoch']
    del setting['test_batch_size']

    hash_key = hashlib.sha1(str(setting).encode()).hexdigest()[:6]
    filename = './results/{}-{}.json'.format(exp_name, hash_key)
    result.update(setting)
    with open(filename, 'w') as f:
        json.dump(result, f)

    
def load_exp_result(exp_name):
    dir_path = './results'
    filenames = [f for f in listdir(dir_path) if isfile(join(dir_path, f)) if '.json' in f]
    list_result = []
    for filename in filenames:
        if exp_name in filename:
            with open(join(dir_path, filename), 'r') as infile:
                results = json.load(infile)
                list_result.append(results)
    df = pd.DataFrame(list_result) # .drop(columns=[])
    return df

In [17]:
def plot_acc(var1, var2, df):

    fig, ax = plt.subplots(1, 3)
    fig.set_size_inches(15, 6)
    sns.set_style("darkgrid", {"axes.facecolor": ".9"})

    sns.barplot(x=var1, y='train_acc', hue=var2, data=df, ax=ax[0])
    sns.barplot(x=var1, y='val_acc', hue=var2, data=df, ax=ax[1])
    sns.barplot(x=var1, y='test_acc', hue=var2, data=df, ax=ax[2])
    
    ax[0].set_title('Train Accuracy')
    ax[1].set_title('Validation Accuracy')
    ax[2].set_title('Test Accuracy')

    
def plot_loss_variation(var1, var2, df, **kwargs):

    list_v1 = df[var1].unique()
    list_v2 = df[var2].unique()
    list_data = []

    for value1 in list_v1:
        for value2 in list_v2:
            row = df.loc[df[var1]==value1]
            row = row.loc[df[var2]==value2]

            train_losses = list(row.train_losses)[0]
            val_losses = list(row.val_losses)[0]

            for epoch, train_loss in enumerate(train_losses):
                list_data.append({'type':'train', 'loss':train_loss, 'epoch':epoch, var1:value1, var2:value2})
            for epoch, val_loss in enumerate(val_losses):
                list_data.append({'type':'val', 'loss':val_loss, 'epoch':epoch, var1:value1, var2:value2})

    df = pd.DataFrame(list_data)
    g = sns.FacetGrid(df, row=var2, col=var1, hue='type', **kwargs)
    g = g.map(plt.plot, 'epoch', 'loss', marker='.')
    g.add_legend()
    g.fig.suptitle('Train loss vs Val loss')
    plt.subplots_adjust(top=0.89) # 만약 Title이 그래프랑 겹친다면 top 값을 조정해주면 됩니다! 함수 인자로 받으면 그래프마다 조절할 수 있겠죠?


def plot_acc_variation(var1, var2, df, **kwargs):
    list_v1 = df[var1].unique()
    list_v2 = df[var2].unique()
    list_data = []

    for value1 in list_v1:
        for value2 in list_v2:
            row = df.loc[df[var1]==value1]
            row = row.loc[df[var2]==value2]

            train_accs = list(row.train_accs)[0]
            val_accs = list(row.val_accs)[0]
            test_acc = list(row.test_acc)[0]

            for epoch, train_acc in enumerate(train_accs):
                list_data.append({'type':'train', 'Acc':train_acc, 'test_acc':test_acc, 'epoch':epoch, var1:value1, var2:value2})
            for epoch, val_acc in enumerate(val_accs):
                list_data.append({'type':'val', 'Acc':val_acc, 'test_acc':test_acc, 'epoch':epoch, var1:value1, var2:value2})

    df = pd.DataFrame(list_data)
    g = sns.FacetGrid(df, row=var2, col=var1, hue='type', **kwargs)
    g = g.map(plt.plot, 'epoch', 'Acc', marker='.')

    def show_acc(x, y, metric, **kwargs):
        plt.scatter(x, y, alpha=0.3, s=1)
        metric = "Test Acc: {:1.3f}".format(list(metric.values)[0])
        plt.text(0.05, 0.95, metric,  horizontalalignment='left', verticalalignment='center', transform=plt.gca().transAxes, bbox=dict(facecolor='yellow', alpha=0.5, boxstyle="round,pad=0.1"))
    g = g.map(show_acc, 'epoch', 'Acc', 'test_acc')

    g.add_legend()
    g.fig.suptitle('Train Accuracy vs Val Accuracy')
    plt.subplots_adjust(top=0.89)

In [18]:
# # ====== Random Seed Initialization ====== #
# seed = 123
# np.random.seed(seed)
# torch.manual_seed(seed)

# parser = argparse.ArgumentParser()
# args = parser.parse_args("")
# args.exp_name = "exp1_lr_model_code"

# # ====== Model ====== #z
# args.out_dim = 10
# args.act = 'relu'

# # ====== Regularization ======= #
# args.l2 = 0.00001

# # ====== Optimizer & Training ====== #
# args.optim = 'RMSprop' #'RMSprop' #SGD, RMSprop, ADAM...
# args.lr = 0.0015
# args.epoch = 10

# args.train_batch_size = 512
# args.test_batch_size = 1024

# # ====== Experiment Variable ====== #
# name_var1 = 'lr'
# name_var2 = 'l2'
# list_var1 = [0.0001]
# list_var2 = [0.00001]


# for var1 in list_var1:
#     for var2 in list_var2:
#         setattr(args, name_var1, var1)
#         setattr(args, name_var2, var2)
#         print(args)
                
#         setting, result = experiment(partition, deepcopy(args))
#         save_exp_result(setting, result)

#         # 오래걸리는 이유
#         # cuda에 안올려서 GPU를 사용안하고 있다
#         # maxpooling을 한번밖에 안해서 parameter가 너무 크다 --> 깊게 쌓고 maxpooling을 여러번 써서 fc에 넣는 차원을 줄이자

In [19]:
# ====== Random Seed Initialization ====== #
seed = 123
np.random.seed(seed)
torch.manual_seed(seed)

parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.exp_name = "exp1_lr_model_code"

# ====== Model ====== #
args.model_code = 'VGG11'
args.in_channels = 3
args.out_dim = 10
args.act = 'relu'

# ====== Regularization ======= #
args.l2 = 0.00001
args.use_bn = True

# ====== Optimizer & Training ====== #
args.optim = 'RMSprop' #'RMSprop' #SGD, RMSprop, ADAM...
args.lr = 0.0015
args.epoch = 10

args.train_batch_size = 512
args.test_batch_size = 1024

# ====== Experiment Variable ====== #
name_var1 = 'lr'
name_var2 = 'model_code'
list_var1 = [0.0001, 0.00001]
list_var2 = ['VGG11', 'VGG13']


for var1 in list_var1:
    for var2 in list_var2:
        setattr(args, name_var1, var1)
        setattr(args, name_var2, var2)
        print(args)
                
        setting, result = experiment(partition, deepcopy(args))
        # save_exp_result(setting, result)

        # 오래걸리는 이유
        # cuda에 안올려서 GPU를 사용안하고 있다
        # maxpooling을 한번밖에 안해서 parameter가 너무 크다 --> 깊게 쌓고 maxpooling을 여러번 써서 fc에 넣는 차원을 줄이자

Namespace(act='relu', epoch=10, exp_name='exp1_lr_model_code', in_channels=3, l2=1e-05, lr=0.0001, model_code='VGG11', optim='RMSprop', out_dim=10, test_batch_size=1024, train_batch_size=512, use_bn=True)
Epoch 0, Acc(train/val): 38.16/32.39, Loss(train/val) 1.59/2.10. Took 10.47 sec
Epoch 1, Acc(train/val): 51.76/53.82, Loss(train/val) 1.28/1.26. Took 10.52 sec
Epoch 2, Acc(train/val): 60.92/56.18, Loss(train/val) 1.07/1.23. Took 10.48 sec
Epoch 3, Acc(train/val): 66.51/57.82, Loss(train/val) 0.93/1.21. Took 10.52 sec
Epoch 4, Acc(train/val): 70.65/61.85, Loss(train/val) 0.81/1.14. Took 10.38 sec
Epoch 5, Acc(train/val): 74.33/67.58, Loss(train/val) 0.73/0.94. Took 10.38 sec
Epoch 6, Acc(train/val): 77.33/70.36, Loss(train/val) 0.64/0.88. Took 10.33 sec
Epoch 7, Acc(train/val): 80.15/70.50, Loss(train/val) 0.57/0.88. Took 10.28 sec
Epoch 8, Acc(train/val): 81.79/69.39, Loss(train/val) 0.53/0.98. Took 10.27 sec
Epoch 9, Acc(train/val): 84.47/72.94, Loss(train/val) 0.45/0.87. Took 10.27